This notebook covers `Train.ipynb` up to the training step.

It doesn't use any of the `category_classification` functions so the example is standalone. \
(I did copy-paste a few functions under the "Helper functions" section, so we should still have a lib)

In [1]:
import sys
sys.path.append('../') #append a relative path to the top package to the search path

In [2]:
from collections import Counter
import dacite
import datetime
import itertools
import json
import numpy as np

import tensorflow as tf
from tensorflow.keras import callbacks, layers
import tensorflow_datasets.public_api as tfds

from category_classification.config import Config

2022-05-15 15:35:01.952594: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-15 15:35:01.952616: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/gabriel/.pyenv/versions/3.8.13/envs/venv-off-tf28/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Build tfds dataset

In [3]:
# Run this once to fetch and build the dataset:

import datasets.off_category

builder = tfds.builder('off_category')
builder.download_and_prepare()

# Or run via command line (if tfds is in the path):
# !cd ../datasets && tfds build off_category

2022-05-13 00:12:30.278710: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...:   0%|                                     | 0/3 [01:22<?, ? url/s]


Dl Completed...:  67%|███████████████████▎         | 2/3 [02:35<00:49, 49.27s/ url]


Dl Completed...:  67%|███████████████████▎         | 2/3 [03:50<00:49, 49.27s/ url]


Dl Completed...:  67%|███████████████████▎         | 2/3 [05:05<00:49, 49.27s/ url]


...

Generating train examples...: 317115 examples [00:44, 6562.17 examples/s]
Generating train examples...: 317817 examples [00:44, 6696.92 examples/s]
Generating train examples...: 318522 examples [00:44, 6801.67 examples/s]
Generating train examples...: 319203 examples [00:45, 6719.54 examples/s]
Generating train examples...: 319876 examples [00:45, 6668.46 examples/s]
Generating train examples...: 320583 examples [00:45, 6786.78 examples/s]
Generating train examples...: 321302 examples [00:45, 6905.21 examples/s]
Generating train examples...: 322006 examples [00:45, 6944.19 examples/s]
Generating train examples...: 322701 examples [00:45, 6762.82 examples/s]
Generating train examples...: 323379 examples [00:45, 6675.67 examples/s]
Generating train examples...: 324048 examples [00:45, 6669.66 examples/s]
Generating train examples...: 324716 examples [00:45, 6633.02 examples/s]
Generating train examples...: 325403 examples [00:45, 6700.38 examples/s]
Generating train examples...: 326077 e

Generating train examples...: 458972 examples [01:06, 6615.59 examples/s]
Generating train examples...: 459634 examples [01:06, 6572.01 examples/s]
Generating train examples...: 460302 examples [01:06, 6603.74 examples/s]
Generating train examples...: 460980 examples [01:06, 6654.05 examples/s]
Generating train examples...: 461658 examples [01:06, 6687.68 examples/s]
Generating train examples...: 462329 examples [01:06, 6694.22 examples/s]
Generating train examples...: 463013 examples [01:06, 6735.34 examples/s]
Generating train examples...: 463687 examples [01:06, 6663.12 examples/s]
Generating train examples...: 464354 examples [01:07, 6650.40 examples/s]
Generating train examples...: 465020 examples [01:07, 6651.64 examples/s]
Generating train examples...: 465688 examples [01:07, 6658.24 examples/s]
Generating train examples...: 466354 examples [01:07, 6656.17 examples/s]
Generating train examples...: 467028 examples [01:07, 6680.89 examples/s]
Generating train examples...: 467697 e

Generating train examples...: 599353 examples [01:27, 6251.09 examples/s]
Generating train examples...: 599981 examples [01:27, 6258.64 examples/s]
Generating train examples...: 600633 examples [01:27, 6335.04 examples/s]
Generating train examples...: 601267 examples [01:27, 6274.74 examples/s]
Generating train examples...: 601895 examples [01:28, 6254.36 examples/s]
Generating train examples...: 602541 examples [01:28, 6311.45 examples/s]
Generating train examples...: 603185 examples [01:28, 6348.83 examples/s]
Generating train examples...: 603834 examples [01:28, 6389.04 examples/s]
Generating train examples...: 604480 examples [01:28, 6406.26 examples/s]
Generating train examples...: 605121 examples [01:28, 6386.34 examples/s]
Generating train examples...: 605760 examples [01:28, 6386.40 examples/s]
Generating train examples...: 606399 examples [01:28, 6324.88 examples/s]
Generating train examples...: 607032 examples [01:28, 6318.20 examples/s]
Generating train examples...: 607672 e

Generating val examples...: 36336 examples [00:04, 7533.64 examples/s]
Generating val examples...: 37090 examples [00:04, 7465.65 examples/s]
Generating val examples...: 37837 examples [00:04, 7428.20 examples/s]
Generating val examples...: 38581 examples [00:05, 7398.66 examples/s]
Generating val examples...: 39338 examples [00:05, 7444.33 examples/s]
Generating val examples...: 40083 examples [00:05, 7438.27 examples/s]
Generating val examples...: 40847 examples [00:05, 7496.99 examples/s]
Generating val examples...: 41626 examples [00:05, 7581.52 examples/s]
Generating val examples...: 42408 examples [00:05, 7652.37 examples/s]
Generating val examples...: 43188 examples [00:05, 7694.75 examples/s]
Generating val examples...: 43964 examples [00:05, 7711.74 examples/s]
Generating val examples...: 44736 examples [00:05, 7548.84 examples/s]
Generating val examples...: 45492 examples [00:05, 7509.76 examples/s]
Generating val examples...: 46254 examples [00:06, 7540.79 examples/s]
Genera

Dataset off_category downloaded and prepared to /home/gabriel/tensorflow_datasets/off_category/0.0.1. Subsequent calls will reuse this data.


# Helper functions

In [4]:
# Copy-pasted from category_classification.data_utils to show how tdfs is used

import pathlib
from typing import List

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

def load_dataset(
        name: str,
        split: str,
        features: List[str],
        labels: str,
        labels_vocab: List[str],
        batch_size: int):

    label_lookup = tf.keras.layers.StringLookup(
            vocabulary = labels_vocab,
            output_mode = 'multi_hot',
            num_oov_indices = 1)

    @tf.function
    def _to_xy(x):
        return ({f: x[f] for f in features}, x[labels])

    @tf.function
    def _label_multi_hot(x, y):
        y = label_lookup(y)
        y = y[1:]  # drop OOV inserted by StringLookup
        return (x, y)

    @tf.function
    def _has_label(x, y):
        return tf.math.reduce_max(y, 0) > 0

    ds = (
        tfds.load(name, split=split)
        .map(_to_xy, num_parallel_calls=tf.data.AUTOTUNE, deterministic=True)
        .map(_label_multi_hot, num_parallel_calls=tf.data.AUTOTUNE, deterministic=True)
        .filter(_has_label)
    )

    if batch_size > 1:
        ds = ds.padded_batch(batch_size)

    return ds


def _flatten_tensor(feature_batch):
    vals = np.hstack(feature_batch.numpy())
    return vals[vals != b'']


def _batch_flatten(ds: tf.data.Dataset, batch_size: int):
    return (
        ds
        .apply(tf.data.experimental.dense_to_ragged_batch(batch_size))
        .map(lambda x: x.merge_dims(0, 1))
    )


def _get_vocabulary(ds: tf.data.Dataset, min_freq: int =1):
    counter = Counter()
    for batch in ds:
        counter.update(batch.numpy())
    return [
        x[0].decode()
        for x in itertools.takewhile(lambda x: x[1] >= min_freq, counter.most_common())
    ]


def _get_feature(ds: tf.data.Dataset, feature_name: str):
    return ds.map(lambda x: x[feature_name])

# Config

In [5]:
with open('../config.json') as f:
    config = dacite.from_dict(Config, json.load(f))

train_config = config.train_config
model_config = config.model_config

preproc_batch_size = 10_000  # some large value

TRAIN_SPLIT = 'train'
VAL_SPLIT = 'val'
TEST_SPLIT = 'test'

# On new data we'll only have a 'train' dataset (by convention TFDS uses 'train' when there's no splits).
# So we'll do the splits with e.g.:

# TRAIN_SPLIT = 'train[:80%]'
# VAL_SPLIT = 'train[80%:90%]'
# TEST_SPLIT = 'train[90%:]'

# ONLY FOR DEMO PURPOSE, so training goes faster
TRAIN_SPLIT = 'train[:10%]'
VAL_SPLIT = 'val[:10%]'
train_config.epochs = 3

# Existing model

In [6]:
tf.random.set_seed(42)

In [7]:
FEATURES = ['product_name', 'ingredients_tags']
LABELS = 'categories_tags'

## Preprocessing

In [8]:
%%time

# FIXME: Dataset here and datasets during train/predict don't go through the same loading
# path nor have the same format. I need to improve the wrappers (load_dataset).
# That said, calling `load` on the same dataset split is virtually free, so it's a matter
# of usability, not perfs.
ds = tfds.load('off_category', split=TRAIN_SPLIT)

# Feature: product_name

product_name_vectorizer_base = tf.keras.layers.TextVectorization(
    split = 'whitespace',
    max_tokens = model_config.product_name_max_tokens,
    output_sequence_length = model_config.product_name_max_length,
)
product_name_vectorizer_base.adapt(
    _get_feature(ds, 'product_name').batch(preproc_batch_size))

# Feature: ingredients_tags

ingredients_vocab_base = _get_vocabulary(
    _batch_flatten(_get_feature(ds, 'ingredients_tags'), batch_size=preproc_batch_size),
    min_freq = model_config.ingredient_min_count)

# Output: categories_tags

categories_vocab_base = _get_vocabulary(
    _batch_flatten(_get_feature(ds, 'categories_tags'), batch_size=preproc_batch_size),
    min_freq = model_config.category_min_count)

2022-05-15 15:35:03.308374: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-15 15:35:03.308397: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-15 15:35:03.308409: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (alba): /proc/driver/nvidia/version does not exist
2022-05-15 15:35:03.308581: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


CPU times: user 9.66 s, sys: 4.49 s, total: 14.1 s
Wall time: 2.47 s


## Model

In [9]:
# The OffCategory tfds dataset generates a dict-based tf.data.Dataset, meaning features
# can be accessed by key instead of position in the inputs. Tensorflow will automatically
# map a feature to its corresponding Input when the feature name matches the Input name.
# It'll ignore features in the dataset that don't match any Input (with a warning), so
# it's much safer than using positions.

# Product name subgraph
product_name_input_base = tf.keras.Input(shape=(1,), dtype=tf.string, name='product_name')

x = product_name_vectorizer_base(product_name_input_base)

x = layers.Embedding(
    input_dim = product_name_vectorizer_base.vocabulary_size(),
    output_dim = model_config.product_name_embedding_size,
    mask_zero = False)(x)

product_name_graph_base = layers.Bidirectional(layers.LSTM(
    units = model_config.product_name_lstm_units,
    recurrent_dropout = model_config.product_name_lstm_recurrent_dropout,
    dropout = model_config.product_name_lstm_dropout))(x)

# Ingredients subgraph
ingredients_input_base = tf.keras.Input(shape=(None,), dtype=tf.string, name='ingredients_tags')

ingredients_graph_base = layers.StringLookup(
    vocabulary=ingredients_vocab_base, output_mode='multi_hot')(ingredients_input_base)

# Model
inputs_base = [product_name_input_base, ingredients_input_base]
graph_inputs_base = [product_name_graph_base, ingredients_graph_base]

x = layers.Concatenate()(graph_inputs_base)
x = layers.Dropout(model_config.hidden_dropout)(x)
x = layers.Dense(model_config.hidden_dim)(x)
x = layers.Dropout(model_config.hidden_dropout)(x)
x = layers.Activation('relu')(x)
output_base = layers.Dense(len(categories_vocab_base), activation='sigmoid')(x)

model_base = tf.keras.Model(inputs=inputs_base, outputs=[output_base])

model_base.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=train_config.lr),
    loss = tf.keras.losses.BinaryCrossentropy(label_smoothing=train_config.label_smoothing),
    metrics = ['binary_accuracy', 'Precision', 'Recall']
)

In [10]:
model_base.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 product_name (InputLayer)      [(None, 1)]          0           []                               
                                                                                                  
 text_vectorization (TextVector  (None, 30)          0           ['product_name[0][0]']           
 ization)                                                                                         
                                                                                                  
 embedding (Embedding)          (None, 30, 64)       1759808     ['text_vectorization[0][0]']     
                                                                                                  
 ingredients_tags (InputLayer)  [(None, None)]       0           []                           

## Training

Expected results:

Epoch 1/3 \
547/547 [==============================] - 28s 47ms/step - loss: 0.0389 - binary_accuracy: 0.9923 - precision: 0.0038 - recall: 0.0064 - val_loss: 0.0139 - val_binary_accuracy: 0.9971 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 \
Epoch 2/3 \
547/547 [==============================] - 25s 47ms/step - loss: 0.0140 - binary_accuracy: 0.9971 - precision: 0.5957 - recall: 0.0044 - val_loss: 0.0133 - val_binary_accuracy: 0.9972 - val_precision: 0.5882 - val_recall: 0.0105 \
Epoch 3/3 \
547/547 [==============================] - 29s 52ms/step - loss: 0.0133 - binary_accuracy: 0.9972 - precision: 0.6496 - recall: 0.0281 - val_loss: 0.0125 - val_binary_accuracy: 0.9972 - val_precision: 0.7539 - val_recall: 0.0478

In [11]:
%%time

!rm -rf cache && mkdir cache

# This is the other way to load the dataset, which I'll try to refactor to also
# support the preprocessing `load` usecase.
# Biggest difference between the two:
# - preprocessing expects a flat dataset format:
#     {f1: Tensor(f1), f2: Tensor(f2), label: Tensor(label)}
# - training expects a "supervised" (features, label) format:
#     ({f1: Tensor(f1), f2: Tensor(f2), ...}, Tensor(label))

ds_train = (
    load_dataset('off_category', TRAIN_SPLIT, FEATURES, LABELS, categories_vocab_base, train_config.batch_size)
    .cache('cache/train')
)
ds_val = (
    load_dataset('off_category', VAL_SPLIT, FEATURES, LABELS, categories_vocab_base, train_config.batch_size)
    .cache('cache/val')
)

model_base.fit(
    ds_train,
    epochs = train_config.epochs,
    validation_data = ds_val,
    callbacks = [
        callbacks.TerminateOnNaN(),
        callbacks.EarlyStopping(monitor='val_loss', patience=4)
    ]
)

Epoch 1/3


2022-05-15 15:35:06.407225: W tensorflow/core/kernels/data/cache_dataset_ops.cc:233] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


    546/Unknown - 27s 45ms/step - loss: 0.0389 - binary_accuracy: 0.9923 - precision: 0.0038 - recall: 0.0064

2022-05-15 15:35:33.466503: W tensorflow/core/kernels/data/cache_dataset_ops.cc:233] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


547/547 [==============================] - 28s 47ms/step - loss: 0.0389 - binary_accuracy: 0.9923 - precision: 0.0038 - recall: 0.0064 - val_loss: 0.0139 - val_binary_accuracy: 0.9971 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/3
  3/547 [..............................] - ETA: 23s - loss: 0.0145 - binary_accuracy: 0.9971 - precision: 0.0000e+00 - recall: 0.0000e+00

2022-05-15 15:35:34.719037: W tensorflow/core/kernels/data/cache_dataset_ops.cc:233] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2022-05-15 15:35:34.738005: W tensorflow/core/kernels/data/cache_dataset_ops.cc:233] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


547/547 [==============================] - 26s 47ms/step - loss: 0.0140 - binary_accuracy: 0.9971 - precision: 0.5957 - recall: 0.0044 - val_loss: 0.0133 - val_binary_accuracy: 0.9972 - val_precision: 0.5882 - val_recall: 0.0105
Epoch 3/3
547/547 [==============================] - 29s 53ms/step - loss: 0.0133 - binary_accuracy: 0.9972 - precision: 0.6496 - recall: 0.0281 - val_loss: 0.0125 - val_binary_accuracy: 0.9972 - val_precision: 0.7539 - val_recall: 0.0478
CPU times: user 6min 50s, sys: 45 s, total: 7min 35s
Wall time: 1min 23s


# Per-feature model construction

This is just moving preprocessing and early parts of the model construction around. I think it gets closer to what Hui Guan has been doing, while still keeping the "raw fields" as inputs to the model, and letting us serialize preproc+model together.

In [12]:
tf.random.set_seed(42)

In [13]:
features = []
labels = None

inputs = []
graph_inputs = []

## Preprocessing

In [14]:
ds = tfds.load('off_category', split=TRAIN_SPLIT)

In [15]:
%%time

feature_name = 'product_name'

product_name_input = tf.keras.Input(shape=(1,), dtype=tf.string, name=feature_name)

product_name_vectorizer = tf.keras.layers.TextVectorization(
    split = 'whitespace',
    max_tokens = model_config.product_name_max_tokens,
    output_sequence_length = model_config.product_name_max_length,
)
product_name_vectorizer.adapt(
    _get_feature(ds, feature_name).batch(preproc_batch_size))

x = product_name_vectorizer(product_name_input)

x = layers.Embedding(
    input_dim = product_name_vectorizer.vocabulary_size(),
    output_dim = model_config.product_name_embedding_size,
    mask_zero = False)(x)

product_name_graph = layers.Bidirectional(layers.LSTM(
    units = model_config.product_name_lstm_units,
    recurrent_dropout = model_config.product_name_lstm_recurrent_dropout,
    dropout = model_config.product_name_lstm_dropout))(x)

features.append(feature_name)
inputs.append(product_name_input)
graph_inputs.append(product_name_graph)

CPU times: user 5.26 s, sys: 2.72 s, total: 7.98 s
Wall time: 1.61 s


In [16]:
%%time

feature_name = 'ingredients_tags'

ingredients_input = tf.keras.Input(shape=(None,), dtype=tf.string, name=feature_name)

ingredients_vocab = _get_vocabulary(
    _batch_flatten(_get_feature(ds, feature_name), batch_size=preproc_batch_size),
    min_freq = model_config.ingredient_min_count)

ingredients_graph = layers.StringLookup(
    vocabulary=ingredients_vocab, output_mode='multi_hot')(ingredients_input)

features.append(feature_name)
inputs.append(ingredients_input)
graph_inputs.append(ingredients_graph)

CPU times: user 6.63 s, sys: 3.43 s, total: 10.1 s
Wall time: 1.67 s


In [17]:
%%time

labels = 'categories_tags'

categories_vocab = _get_vocabulary(
    _batch_flatten(_get_feature(ds, labels), batch_size=preproc_batch_size),
    min_freq = model_config.category_min_count)

# StringLookup(output_mode='multi_hot') mode requires num_oov_indices >= 1.
# We don't want OOVs in the categories_tags output layer, since it wouldn't make
# sense to predict OOV. So we'll drop the OOV in _transform below.
# Be careful when using StringLookup methods, some of them will return values
# based on a vocabulary with OOV (e.g. vocabulary_size()). Keep this in mind when
# mapping predictions back to the original vocabulary.
categories_multihot = tf.keras.layers.StringLookup(
    vocabulary=categories_vocab, output_mode='multi_hot', num_oov_indices=1)  

def categories_encode(ds: tf.data.Dataset):
    @tf.function
    @tf.autograph.experimental.do_not_convert
    def _transform(x, y):
        y = categories_multihot(y)
        y = y[1:]  # drop OOV
        return (x, y)

    @tf.function
    @tf.autograph.experimental.do_not_convert
    def _not_empty(x, y):
        return tf.math.reduce_max(y, 0) > 0

    return (
        ds
        .map(_transform, num_parallel_calls=tf.data.AUTOTUNE, deterministic=True)
        .filter(_not_empty)
    )

CPU times: user 6.41 s, sys: 3.55 s, total: 9.96 s
Wall time: 1.66 s


## Model

In [18]:
x = layers.Concatenate()(graph_inputs)
x = layers.Dropout(model_config.hidden_dropout)(x)
x = layers.Dense(model_config.hidden_dim)(x)
x = layers.Dropout(model_config.hidden_dropout)(x)
x = layers.Activation('relu')(x)
output = layers.Dense(len(categories_vocab), activation='sigmoid')(x)

model = tf.keras.Model(inputs=inputs, outputs=[output])

model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=train_config.lr),
    loss = tf.keras.losses.BinaryCrossentropy(label_smoothing=train_config.label_smoothing),
    metrics = ['binary_accuracy', 'Precision', 'Recall']
)

In [19]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 product_name (InputLayer)      [(None, 1)]          0           []                               
                                                                                                  
 text_vectorization_1 (TextVect  (None, 30)          0           ['product_name[0][0]']           
 orization)                                                                                       
                                                                                                  
 embedding_1 (Embedding)        (None, 30, 64)       1759808     ['text_vectorization_1[0][0]']   
                                                                                                  
 ingredients_tags (InputLayer)  [(None, None)]       0           []                         

## Training

In [20]:
%%time

!rm -rf cache && mkdir cache

ds_train = (
    tfds.load('off_category', split=TRAIN_SPLIT, as_supervised=True)
    .apply(categories_encode)
    .padded_batch(train_config.batch_size)
    .cache('cache/train')
)

ds_val = (
    tfds.load('off_category', split=VAL_SPLIT, as_supervised=True)
    .apply(categories_encode)
    .padded_batch(train_config.batch_size)
    .cache('cache/val')
)

model.fit(
    ds_train,
    epochs = train_config.epochs,
    validation_data = ds_val,
    callbacks = [
        callbacks.TerminateOnNaN(),
        callbacks.EarlyStopping(monitor='val_loss', patience=4)
    ]
)

Epoch 1/3


2022-05-15 15:36:35.336330: W tensorflow/core/kernels/data/cache_dataset_ops.cc:233] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
/home/gabriel/.pyenv/versions/3.8.13/envs/venv-off-tf28/lib/python3.8/site-packages/keras/engine/functional.py:559: UserWarning: Input dict contained keys ['code'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


    546/Unknown - 35s 57ms/step - loss: 0.0389 - binary_accuracy: 0.9923 - precision: 0.0038 - recall: 0.0064

2022-05-15 15:37:10.107473: W tensorflow/core/kernels/data/cache_dataset_ops.cc:233] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


547/547 [==============================] - 37s 60ms/step - loss: 0.0389 - binary_accuracy: 0.9923 - precision: 0.0038 - recall: 0.0064 - val_loss: 0.0139 - val_binary_accuracy: 0.9971 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/3
  3/547 [..............................] - ETA: 26s - loss: 0.0145 - binary_accuracy: 0.9971 - precision: 0.0000e+00 - recall: 0.0000e+00

2022-05-15 15:37:11.971324: W tensorflow/core/kernels/data/cache_dataset_ops.cc:233] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2022-05-15 15:37:12.001827: W tensorflow/core/kernels/data/cache_dataset_ops.cc:233] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


547/547 [==============================] - 28s 51ms/step - loss: 0.0140 - binary_accuracy: 0.9971 - precision: 0.5957 - recall: 0.0044 - val_loss: 0.0133 - val_binary_accuracy: 0.9972 - val_precision: 0.5882 - val_recall: 0.0105
Epoch 3/3
547/547 [==============================] - 28s 51ms/step - loss: 0.0133 - binary_accuracy: 0.9972 - precision: 0.6496 - recall: 0.0281 - val_loss: 0.0125 - val_binary_accuracy: 0.9972 - val_precision: 0.7539 - val_recall: 0.0478
CPU times: user 7min 48s, sys: 47.6 s, total: 8min 36s
Wall time: 1min 33s
